# Auxiliares

In [2]:
from scipy.stats import uniform, norm
import numpy as np

In [18]:
MIN_SAMPLES = 100

def print_res(name, nsim, res, var, alfa, real):
    print(f'Funcion {name} | Nsim {nsim}' )
    print(f'Resultados | Real {real:.5} | Diferencia   {abs(real - res)} ')
    print(f'\t| Calculado    {res:.5}') 
    print(f'\t| Varianza     {var:.5}')
    if alfa:
        c = norm.ppf(1-alfa)*(var/nsim)**(1/2)
        print(f'\t| IC(95) =     ({res - c:.5},{res + c:.5})')

def estadistico(fun, d):
    np.random.seed(1000)
    mean = fun(uniform.rvs())
    var = 0
    i = 1
    while (i < MIN_SAMPLES or (var/i)**(1/2) >= d):
        s = fun(uniform.rvs())
        mean_old = mean
        mean = mean_old + (s - mean_old)/(i + 1)
        var = (1 - 1/i)*var + (i+1)*(mean_old - mean)**2
        i += 1
    return i, mean, var

In [4]:
def g_i(x):
    # En [pi,2pi]
    return np.sin(x)/x

def g_i_01(x):
    # En [0,1]
    y = ( x + 1 )*np.pi
    dy = np.pi
    return g_i(y) * dy

def g_ii(x):
    # En [0,inf]
    return 3 / (3+x**4)

def g_ii_01(x):
    # En [0,1]
    y = (1/x - 1)
    dy = 1/(x**2) 
    return g_ii(y) * dy

In [5]:
def montecarlo_01(g, nsim):
    ''' Funcion para el metodo de montecarlo en el intervalo [0, 1]
    '''
    np.random.seed(1000)
    integral = 0
    for u in uniform.rvs(size=nsim):
        integral += g(u)
    return integral/nsim


In [6]:
def estadistico_intervalo(fun, z, l):
    d = l / (2*z)
    return estadistico(fun, d)

# Resultados

In [7]:
r1_m = montecarlo_01(g_i_01, 10_000)
print_res('mtc-gi', 10000, r1_m, 'N/A', None, -1*0.4337854758498377)
r2_m = montecarlo_01(g_ii_01, 10_000)
print_res('mtc-gii', 10000, r2_m, 'N/A', None, 1.46179069437506)

Funcion mtc-gi
	| Real	       -0.4337854758498377
	| Calculado    -0.43355489833898175
	| Diferencia   0.0002305775108559227
	| Simulaciones 10000
	| S            N/A
Funcion mtc-gii
	| Real	       1.46179069437506
	| Calculado    1.4714659759204278
	| Diferencia   0.009675281545367787
	| Simulaciones 10000
	| S            N/A


In [ ]:
r1_e = estadistico(g_i_01, 0.001)
r2_e = estadistico(g_ii_01, 0.001)

In [23]:
print('-------------------------------------------------------------------')
print_res('est-gi', r1_e[0], r1_e[1], r1_e[2], 0.05, -1*0.4337854758498377)
print('-------------------------------------------------------------------')
print_res('est-gii', r2_e[0], r2_e[1], r2_e[2], 0.05, 1.46179069437506)
print('-------------------------------------------------------------------')

-------------------------------------------------------------------
Funcion est-gi | Nsim 44481
Resultados | Real -0.43379 | Diferencia   0.0004387610331271774 
	| Calculado    -0.43335
	| Varianza     0.04448
	| IC(95) =     (-0.43499,-0.4317)
-------------------------------------------------------------------
Funcion est-gii | Nsim 951259
Resultados | Real 1.4618 | Diferencia   0.0027290414822074283 
	| Calculado    1.4645
	| Varianza     0.95126
	| IC(95) =     (1.4629,1.4662)
-------------------------------------------------------------------
